In [1]:
from collections import Counter
from sklearn.model_selection import KFold
import numpy as np
from scipy.sparse import csr_matrix
import re
import os
import shutil

from IPython.display import clear_output
from collections import defaultdict

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

from functions import *

from keras.models import Sequential, Model
from keras.losses import mean_absolute_error
from keras.layers import Embedding, LSTM, Bidirectional, Dense, Input, Masking, Lambda, concatenate
import keras.backend as K
from keras.optimizers import Adam
from keras.callbacks import LambdaCallback, LearningRateScheduler, Callback

Using TensorFlow backend.


In [2]:
QUERY_DICT_SIZE = 19829#247074
DOCUMENT_DICT_SIZE = 262285#583954
ACTIVATION = 'relu'
MAX_DOCUMENT_LEN = 35039#35840
MAX_QUERY_LEN = 1425#1475

In [3]:
dense_model = make_lstm_dense_model(
    DOCUMENT_DICT_SIZE,
    10,
    QUERY_DICT_SIZE,
    2,
    2,
    MAX_QUERY_LEN,
    'tanh',
    HIDDEN_SIZE
)

In [4]:
input_from_lines = lambda lines: make_lstm_dense_input_from_lines(lines, MAX_QUERY_LEN, DOCUMENT_DICT_SIZE)

In [ ]:
history = train_epoch(dense_model, 1, 20, input_from_lines)

Epoch 1/1
   7/7656 [..............................] - ETA: 38:25:15 - loss: 0.0045 - mean_positive_score: 0.0103 - mean_negative_score: 0.0125 - mean_positive_var: 0.0092 - mean_negative_var: 0.0097 - acc: 0.0056         

In [ ]:
with open("positive_train_data.tsv") as positive:
    positive_lines = [next(positive) for i in range(10)]

In [ ]:
with open("negative_train_data.tsv") as negative:
    negative_lines = [next(negative) for i in range(10)]

In [ ]:
positive_inputs = make_lstm_dense_input_from_lines(positive_lines)

In [ ]:
dense_model.predict(positive_inputs)

In [ ]:
negative_inputs = make_lstm_dense_input_from_lines(negative_lines)

In [ ]:
dense_model.predict(negative_inputs)

In [ ]:
test_data_generator = make_generator(DataGenerator(
    "positive_train_data_10K.tsv",
    "negative_train_data_10K.tsv",
    dense_model,
    BATCH_SIZE,
    BATCH_SIZE * 1
))
z = next(test_data_generator)

In [ ]:
x_1 = dense_model.evaluate_generator(test_data_generator, 1)

In [ ]:
x_2 = dense_model.evaluate_generator(test_data_generator, 2)

In [ ]:
x_5 = dense_model.evaluate_generator(test_data_generator, 5)

In [ ]:
x_10 = dense_model.evaluate_generator(test_data_generator, 10)

In [ ]:
x_100 = dense_model.evaluate_generator(test_data_generator, 100)

In [ ]:
x_1

In [ ]:
x_100

In [ ]:
make_batch(positive_lines[0], negative_lines)

In [ ]:
(1 - np.array([ 1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.])) / 2